In [1]:
import itertools

import numpy as np
from scipy.spatial.distance import cdist

import sys
sys.path.append('..')

import analysis.math_funcs as mf
import modules.iterable_funcs as itf
import modules.pose_estimation as pe


In [2]:
def score_func(a, b):
    """Score function for scoring links between body parts."""
    x = 1 / mf.norm_ratio(a, b)
    return -(x - 1)**2 + 1

radii = [i for i in range(0, 50, 5)]

population = np.array([[-115.58048642,   60.89011957,  244.777     ],
       [-116.9785391 ,   20.47026348,  250.67      ],
       [-114.922309  ,   63.3364844 ,  243.795     ],
       [-117.36944869,   30.3507099 ,  247.751     ],
       [-117.52351807,   28.23920431,  243.436     ],
       [-118.19426157,   12.50639743,  245.43      ],
       [-121.35864205,   -0.97322703,  255.433     ],
       [-117.90960175,   59.95409882,  242.64      ],
       [-120.75570116,    1.88955716,  253.303     ],
       [-120.12484905,    4.14457176,  251.536     ],
       [-119.35087127,    5.81290497,  247.594     ],
       [-114.63498171,   68.3179124 ,  241.164     ],
       [-107.48723883,  -12.8211624 ,  230.018     ],
       [-118.43413556,   14.0089565 ,  248.52      ],
       [-106.99350063,  -14.68416389,  229.924     ],
       [-118.24636692,   13.95480618,  248.716     ],
       [-121.42820509,  -31.98467921,  256.824     ],
       [-126.28805758,  -12.45890766,  275.076     ],
       [-120.39244269,  -48.58608126,  259.256     ],
       [-126.28805758,  -12.45890655,  275.076     ],
       [-107.82113617,  -15.6490517 ,  229.868     ],
       [-120.39244269,  -48.58608126,  259.256     ],
       [-107.83374389,  -15.75142919,  229.866     ]])

paths = np.array([[ 2,  4,  5,  8, 16, 17],
       [ 2,  4,  5,  8, 16, 18],
       [ 2,  4,  5,  8, 16, 19],
       [ 2,  4,  5,  8, 16, 20],
       [ 2,  4,  5,  8, 16, 21],
       [ 2,  4,  5,  8, 16, 22]])

label_adj_list = {0: {1: 69.34934638727614},
 1: {2: 21.99911000126585, 3: 38.764894273373315},
 2: {3: 16.76578427210746},
 3: {4: 24.624453515469536, 5: 47.98735770357412},
 4: {5: 23.362904188104586},
 5: {}}

In [3]:
def reduce_population(population, paths):
    
    path_nums = np.unique(paths)

    # Population along the shortest paths
    pop_reduced = population[path_nums, :]

    n_pop = len(path_nums)
    mapping = {k: v for k, v in zip(path_nums, range(n_pop))}

    paths_reduced = np.zeros(paths.shape, dtype=int)
    n_paths, n_types = paths.shape
    
    for i in range(n_paths):
        for j in range(n_types):
            paths_reduced[i, j] = mapping[paths[i, j]]
            
    return pop_reduced, paths_reduced

In [4]:
def get_path_vectors(paths, n_pop):
    
    n_paths = paths.shape[0]
    path_vectors = np.zeros((n_paths, n_pop))

    all_nums = [i for i in range(n_pop)]
    for i, path in enumerate(paths):
        path_vectors[i, :] = np.in1d(all_nums, path)
        
    return path_vectors

In [5]:
def select_best_feet(dist_matrix, score_matrix, path_vectors, radii):

    n_paths = path_vectors.shape[0]
    
    combos = [*itertools.combinations(range(n_paths), 2)]
    n_combos = len(combos)

    votes, combo_scores = np.zeros(n_combos), np.zeros(n_combos)

    for r in radii:

        mask = dist_matrix < r
        scores_of_radius = mask * score_matrix

        for i in range(n_combos):

            a, b = combos[i]

            path_1 = path_vectors[a, :]
            path_2 = path_vectors[b, :]

            path_joined = np.logical_or(path_1, path_2)

            combo_scores[i] = np.sum(scores_of_radius[:, path_joined])

        max_score = max(combo_scores)

        # Winning combos for this radius
        radius_winners = combo_scores == max_score

        # Votes go to the winners
        votes += radius_winners

    winning_combo = np.argmax(votes)
    foot_1, foot_2 = combos[winning_combo]

    return foot_1, foot_2

In [6]:
pop_reduced, paths_reduced = reduce_population(population, paths)

dist_matrix = cdist(pop_reduced, pop_reduced)
score_matrix = pe.get_scores(dist_matrix, paths_reduced, label_adj_list, score_func)

path_vectors = get_path_vectors(paths_reduced, pop_reduced.shape[0])

foot_1, foot_2 = select_best_feet(dist_matrix, score_matrix, path_vectors, radii)

foot_1, foot_2

(3, 5)